In [2]:
from evo.tools import log
log.configure_logging()
from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation
from evo.core.units import Unit
from evo.tools.settings import SETTINGS
from rclpy.serialization import deserialize_message
from visualization_msgs.msg import MarkerArray, Marker
from tf2_msgs.msg import TFMessage
from sensor_msgs.msg import PointCloud2
from evo.tools import file_interface
from rosbags.rosbag2 import Reader as Rosbag2Reader
from rosbags.serde import deserialize_cdr
import math
import numpy as np
from scipy.spatial.transform import Rotation
from pathlib import Path
import os
from rosbags.typesys import get_types_from_idl, get_types_from_msg, register_types
import copy
from rosbags.rosbag2 import Writer as Rosbag2Writer
from rclpy.serialization import serialize_message
from rosbags.typesys.types import sensor_msgs__msg__CompressedImage, sensor_msgs__msg__CameraInfo, tf2_msgs__msg__TFMessage
from cv_bridge import CvBridge
import cv2
from sensor_msgs.msg import CameraInfo
from geometry_msgs.msg import TransformStamped
from rosbags.serde import serialize_cdr

/tmp/ipykernel_4594/4085804303.py:24: DeprecationWarning: 
Importing from 'rosbags.typesys.types' is deprecated.
Use a specific type store instead, for example:
from rosbags.typesys.stores.ros2_foxy import std_msgs__msg__Header

  from rosbags.typesys.types import sensor_msgs__msg__CompressedImage, sensor_msgs__msg__CameraInfo, tf2_msgs__msg__TFMessage


In [29]:
type_converter = {
    "visualization_msgs/msg/MarkerArray": MarkerArray,
    "visualization_msgs/msg/Marker": Marker,
    "sensor_msgs/msg/PointCloud2": PointCloud2
}

bag_name = "/home/joshuabird/Desktop/apr13_lab_dataset_processed"
agent_ids=[1,2,3]

# if directory exists, remove it
if os.path.exists(f"{bag_name}_viz_processed"):
    os.system(f"rm -r {bag_name}_viz_processed")

with Rosbag2Reader(bag_name) as reader:
    with Rosbag2Writer(f"{bag_name}_viz_processed") as writer:
        # Read visuzalization data
        robot_names = [f"robot{agent_id}" for agent_id in agent_ids]
        topics = [f'/{robot_name}/all_points' for robot_name in robot_names] + \
                [f'/{robot_name}/kf_markers' for robot_name in robot_names] + \
                [f'/{robot_name}/camera_pose_marker' for robot_name in robot_names] + \
                [f'/{robot_name}/tracked_points' for robot_name in robot_names] + \
                ["/tf"]
        
        new_connections = {}
        for connection in reader.connections:
            if connection.topic in topics:
                new_connections[connection] = writer.add_connection(connection.topic, connection.msgtype)

        # create a static transform to "world" frame that rotates around x axis by -15 degrees
        transform = TransformStamped()
        transform.header.frame_id = "world_rotated"
        transform.child_frame_id = "world"
        transform.transform.rotation.x = np.sin(-18/2 * np.pi/180)
        transform.transform.rotation.y = 0
        transform.transform.rotation.z = 0
        transform.transform.rotation.w = np.cos(-18/2 * np.pi/180)
        tf_msg = TFMessage()
        tf_msg.transforms.append(transform)
                
        sent_static = False
        connections = [c for c in reader.connections if c.topic in topics]
        for connection, timestamp, rawdata in reader.messages(connections=connections):
            if not sent_static:
                writer.write(new_connections[[c for c in reader.connections if c.topic=="/tf"][0]], timestamp, serialize_cdr(tf_msg, tf2_msgs__msg__TFMessage.__msgtype__))
                sent_static = True

            if connection.topic in [f'/{robot_name}/kf_markers' for robot_name in robot_names]:
                type_def = type_converter[connection.msgtype]
                data = deserialize_message(rawdata, type_def)

                for marker in data.markers:
                    scale = 1.

                    if marker.ns == "keyFrameWireframes":

                        marker.scale.x *= scale * 1.8
                        marker.scale.y *= scale * 1.8
                        marker.scale.z *= scale * 1.8

                        for point in marker.points:
                            point.x *= scale
                            point.y *= scale
                            point.z *= scale

                        if marker.color.r == 0.5:
                            marker.color.r = 255/255
                            marker.color.g = 72/255
                            marker.color.b = 72/255
                            marker.color.a = 1.
                        if marker.color.g == 0.5:
                            marker.color.r = 72/255
                            marker.color.g = 230/255
                            marker.color.b = 72/255
                            marker.color.a = 1.
                        
                        # marker.color.r = 72/255
                        # marker.color.g = 230/255
                        # marker.color.b = 72/255
                        # marker.color.a = 1.

                        # if connection.topic.split("/")[1] == "robot1":
                        #     marker.color.r = 255/255
                        #     marker.color.g = 72/255
                        #     marker.color.b = 72/255
                        #     marker.color.a = 1.

                    if marker.ns == "connectedKeyFrameLines":
                        marker.color.r = 0.
                        marker.color.g = 0.
                        marker.color.b = 1.
                        marker.color.a = .5

                        marker.scale.x *= scale
                        marker.scale.y *= scale
                        marker.scale.z *= scale


                writer.write(new_connections[connection], timestamp, serialize_message(data))
            else:
                writer.write(new_connections[connection], timestamp, rawdata)


/tmp/ipykernel_4594/2991346701.py:27: DeprecationWarning: Writer.add_connection should be called with typestore or msgdef/rihs01 pair.
  new_connections[connection] = writer.add_connection(connection.topic, connection.msgtype)
/tmp/ipykernel_4594/2991346701.py:44: DeprecationWarning: Global serialization/deserialization functions have been replaced with
explicit typestores.

If you are deserializing messages from an AnyReader instance, simply
use its `.deserialize(data, typename)` method.

Otherwise instantiate a type store and use its methods:

from rosbags.typesys import Stores, get_typestore

typestore = get_typestore(Stores.ROS2_FOXY)
typestore.deserialize_cdr(data, typename)
  writer.write(new_connections[[c for c in reader.connections if c.topic=="/tf"][0]], timestamp, serialize_cdr(tf_msg, tf2_msgs__msg__TFMessage.__msgtype__))


In [4]:
new_connections[[c for c in reader.connections if c.topic=="/tf"][0]]

Connection(id=2, topic='/tf', msgtype='tf2_msgs/msg/TFMessage', msgdef='geometry_msgs/TransformStamped[] transforms\n================================================================================\nMSG: geometry_msgs/TransformStamped\nstd_msgs/Header header\nstring child_frame_id\ngeometry_msgs/Transform transform\n================================================================================\nMSG: std_msgs/Header\nbuiltin_interfaces/Time stamp\nstring frame_id\n================================================================================\nMSG: builtin_interfaces/Time\nint32 sec\nuint32 nanosec\n================================================================================\nMSG: geometry_msgs/Transform\ngeometry_msgs/Vector3 translation\ngeometry_msgs/Quaternion rotation\n================================================================================\nMSG: geometry_msgs/Vector3\nfloat64 x\nfloat64 y\nfloat64 z\n==================================================================